In [35]:
# based off crowsonkb's code
import argparse
from concurrent import futures
from itertools import chain, islice
import json
import os
from pathlib import Path

from transformers import AutoTokenizer
from tqdm import tqdm

In [36]:
def batched(iterable, n):
    "Batch data into tuples of length n. The last batch may be shorter."
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError("n must be at least one")
    it = iter(iterable)
    while batch := tuple(islice(it, n)):
        yield batch

In [37]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [39]:
!pip install pandas
import pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 106.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 47.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [47]:
test_dset = pandas.read_csv('test_dataset.csv')


In [57]:
train_dset = pandas.read_csv('train_dataset.csv')

In [49]:
test_dset.head()

,text,__index_level_0__
0,ELLALALLWLEA ELLA ELLA pokemon is different wt...,2740
1,meeting this tuesday + announce WeChat groups ...,1911
2,😭 it’s ok okok < 😄 hi also we should aim to ge...,1263
3,😭 on the internet man why would ppl lie 😭 THE ...,1456
4,bear and also i did not feel like clicking the...,2254


In [51]:
texts = test_dset['text'].tolist()

In [58]:
texts1 = train_dset['text'].tolist()

In [52]:
#tokenize
with futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    gen = (inputs.input_ids for inputs in executor.map(tokenizer, texts))
    document_tokens = list(tqdm(gen, total=len(texts)))

100%|██████████| 551/551 [00:00<00:00, 38243.61it/s]


In [63]:
with futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    gen1 = (inputs.input_ids for inputs in executor.map(tokenizer, texts1))
    document_tokens1 = list(tqdm(gen1, total=len(texts1)))

100%|██████████| 2203/2203 [00:00<00:00, 47105.87it/s]


In [53]:
#append eos
for tokens in document_tokens:
    tokens.append(tokenizer.eos_token_id)

In [64]:
#append eos
for tokens in document_tokens1:
    tokens.append(tokenizer.eos_token_id)

In [55]:
test_dset['tokenized_texts'] = document_tokens


In [65]:
train_dset['tokenized_texts'] = document_tokens1


In [71]:
dt = document_tokens + document_tokens1

In [74]:
import pickle
with open('dataset.pkl', 'wb') as file:
    pickle.dump(dt, file)


In [69]:
train_dset.to_csv('train_tokenized.csv')

In [70]:
test_dset.to_csv('test_tokenized.csv')